# Multi-step Logistics Scenario
This notebook wires up a shipping-focused tool stack so the agent must combine several tool calls to answer a single request.
Run the cells sequentially and finish by executing the scenario cell to watch the agent plan a shipment.

In [1]:
import json
from typing import Annotated

from httpx import AsyncClient
from loguru import logger
from dotenv import dotenv_values

from aceai.agent import AgentBase, ToolExecutor
from aceai.llm import LLMService
from aceai.llm.openai import OpenAI
from aceai.tools import spec, tool, Tool
from ididi import Graph, use

values = dotenv_values(".env")


In [2]:
WAREHOUSE_ORDERS = {
    "ORD-200": {
        "destination": "Denver, CO",
        "priority": "express",
        "notes": "Helios Labs needs hardware before the Wednesday stand-up.",
        "items": [
            {"sku": "sensor-kit", "quantity": 45},
            {"sku": "control-module", "quantity": 30},
            {"sku": "battery-pack", "quantity": 60},
        ],
    },
    "ORD-207": {
        "destination": "Austin, TX",
        "priority": "standard",
        "notes": "Stocking field pod replacements.",
        "items": [
            {"sku": "telemetry-node", "quantity": 22},
            {"sku": "cooling-shroud", "quantity": 18},
        ],
    },
}

SKU_WEIGHTS = {
    "sensor-kit": 0.85,
    "control-module": 1.4,
    "battery-pack": 0.65,
    "telemetry-node": 1.1,
    "cooling-shroud": 2.2,
}

SHIPPING_RATES = {
    "standard": {"base": 48.0, "per_kg": 1.35, "eta_days": 5},
    "express": {"base": 92.0, "per_kg": 1.95, "eta_days": 2},
}

@tool
def lookup_order(
    order_id: Annotated[str, spec(description="Order identifier such as ORD-200.")]
) -> str:
    """Return line items, destination, and priority for a warehouse order."""
    order = WAREHOUSE_ORDERS.get(order_id.upper())
    if not order:
        raise ValueError(f"Unknown order {order_id}")
    return json.dumps(order)

@tool
def get_sku_weight(
    sku: Annotated[str, spec(description="Catalog SKU to pull the per-unit weight for.")]
) -> str:
    """Return the per-unit weight for a SKU in kilograms."""
    key = sku.lower()
    if key not in SKU_WEIGHTS:
        raise ValueError(f"Unknown SKU {sku}")
    return json.dumps({"sku": key, "weight_kg": SKU_WEIGHTS[key]})

@tool
def estimate_shipping_cost(
    weight_kg: Annotated[float, spec(description="Total shipment mass in kilograms.")],
    method: Annotated[str, spec(description="Shipping tier to price (standard or express).")],
) -> str:
    """Quote the shipping cost given a total weight and service tier."""
    method_key = method.strip().lower()
    if method_key not in SHIPPING_RATES:
        raise ValueError(f"Unsupported shipping method {method}")
    rates = SHIPPING_RATES[method_key]
    cost = rates["base"] + weight_kg * rates["per_kg"]
    return json.dumps(
        {
            "method": method_key,
            "weight_kg": round(weight_kg, 2),
            "cost_usd": round(cost, 2),
            "eta_days": rates["eta_days"],
        }
    )

OPEN_METEO_GEOCODE = "https://geocoding-api.open-meteo.com/v1/search"
OPEN_METEO_FORECAST = "https://api.open-meteo.com/v1/forecast"

async def build_async_http_client() -> AsyncClient:
    return AsyncClient(timeout=20.0)

@tool
async def fetch_weather_window(
    city: Annotated[str, spec(description="Destination city to inspect")],
    client: Annotated[AsyncClient, use(build_async_http_client, reuse=False)],
) -> str:
    """Fetch a quick temperature and precipitation outlook for the next 24 hours."""
    try:
        geo_resp = await client.get(
            OPEN_METEO_GEOCODE,
            params={"name": city, "count": 1, "language": "en", "format": "json"},
            timeout=15.0,
        )
        geo_resp.raise_for_status()
        geo_payload = geo_resp.json()
        results = geo_payload.get("results") or []
        if not results:
            raise ValueError(f"No coordinates found for {city}")
        location = results[0]
        lat = location["latitude"]
        lon = location["longitude"]

        forecast_resp = await client.get(
            OPEN_METEO_FORECAST,
            params={
                "latitude": lat,
                "longitude": lon,
                "hourly": "temperature_2m,precipitation_probability",
                "forecast_days": 1,
                "timezone": "auto",
            },
            timeout=15.0,
        )
        forecast_resp.raise_for_status()
        forecast = forecast_resp.json()["hourly"]
        temps = forecast["temperature_2m"]
        precip = forecast["precipitation_probability"]
        avg_temp = sum(temps) / len(temps)
        max_precip = max(precip)

        summary = {
            "city": city,
            "latitude": lat,
            "longitude": lon,
            "avg_temp_c": round(avg_temp, 1),
            "max_precip_probability": int(max_precip),
            "source": "open-meteo",
        }
        return json.dumps(summary)
    finally:
        await client.aclose()


In [3]:
import sys
import time

from aceai.llm.interface import LLMToolCall

logger.remove()
logger.add(
    sys.stdout,
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level:<7} | {message}",
    colorize=False,
)

class LoggingToolExecutor(ToolExecutor):
    async def execute_tool(self, tool_call: LLMToolCall) -> str:
        call_id = tool_call.call_id or tool_call.id or "n/a"
        logger.info(
            "Tool {name} starting (call_id={call_id})",
            name=tool_call.name,
            call_id=call_id,
        )
        start = time.perf_counter()
        try:
            payload = await super().execute_tool(tool_call)
        except Exception:
            duration = time.perf_counter() - start
            logger.exception(
                "Tool {name} failed after {duration:.2f}s",
                name=tool_call.name,
                duration=duration,
            )
            raise
        duration = time.perf_counter() - start
        logger.success(
            "Tool {name} finished in {duration:.2f}s",
            name=tool_call.name,
            duration=duration,
        )
        return payload


In [4]:
def build_agent(prompt: str, max_turns: int, tools: list[Tool]) -> AgentBase:
    openai_llm = OpenAI(
        api_key=values["OPENAI_API_KEY"],
        default_model="gpt-4",
        default_stream_model="gpt-4-turbo",
    )
    graph = Graph()

    llm_service = LLMService(
        providers=[openai_llm],
        timeout_seconds=120,
    )
    executor = LoggingToolExecutor(
        tools=tools,
        graph=graph,
    )


    return AgentBase(
        prompt=prompt,
        default_model="gpt-4",
        llm_service=llm_service,
        executor=executor,
        max_turns=max_turns,
    )

agent = build_agent(prompt=(
        "You are the logistics coordinator for AceAI. "
        "Always inspect orders, fetch SKU weights, price shipping strictly through the available tools, "
        "and incorporate the weather outlook before deciding on a service level."
    ), max_turns=20, tools=[lookup_order, get_sku_weight, estimate_shipping_cost, fetch_weather_window])


## Scenario: plan shipment for ORD-200
The following cell asks the agent to pull data for an urgent order, compute the shipment mass, price both service tiers, and consider the destination weather. Expect the agent to call:
1. `lookup_order`
2. `get_sku_weight` for each SKU
3. `estimate_shipping_cost` twice
4. `fetch_weather_window` for the destination city

In [5]:
multi_step_question = """
You are preparing a logistics brief for Helios Labs covering order ORD-200.

Tasks:
1. Call `lookup_order` to restate the destination, priority, and every SKU with its quantity.
2. Use `get_sku_weight` for each SKU individually so you can calculate the total shipment weight in kilograms.
3. Price BOTH `standard` and `express` service levels by calling `estimate_shipping_cost` twice with the total weight.
4. Call `fetch_weather_window` for the destination city to understand short-term weather risks that might impact delivery.
5. Recommend which service level to book, citing cost, ETA, the customer's stated priority, and the weather outlook.

Present the answer as a brief overview paragraph plus bullet points that cover total weight, each quote, the weather takeaway, and the final recommendation.
"""

await agent.handle(multi_step_question)


2025-12-06 04:04:09.324 | INFO    | Tool lookup_order starting (call_id=call_zkOF8FHIV4hNCN158XZpp3Qw)
2025-12-06 04:04:09.325 | SUCCESS | Tool lookup_order finished in 0.00s
2025-12-06 04:04:15.081 | INFO    | Tool get_sku_weight starting (call_id=call_hGQ9TB25OVVQnCCQoq3ZInXR)
2025-12-06 04:04:15.082 | SUCCESS | Tool get_sku_weight finished in 0.00s
2025-12-06 04:04:15.082 | INFO    | Tool get_sku_weight starting (call_id=call_S1GUrXveXMEHWS1PWxSClm40)
2025-12-06 04:04:15.083 | SUCCESS | Tool get_sku_weight finished in 0.00s
2025-12-06 04:04:15.083 | INFO    | Tool get_sku_weight starting (call_id=call_KNixyvMVnFBNrVkbX5I6Nsip)
2025-12-06 04:04:15.084 | SUCCESS | Tool get_sku_weight finished in 0.00s
2025-12-06 04:04:17.684 | INFO    | Tool estimate_shipping_cost starting (call_id=call_mMC9cmSagakY2Q7kZwm3AeQM)
2025-12-06 04:04:17.684 | SUCCESS | Tool estimate_shipping_cost finished in 0.00s
2025-12-06 04:04:20.453 | INFO    | Tool estimate_shipping_cost starting (call_id=call_S1N92G

ValueError: No coordinates found for Denver, CO